# towards Training

1. existing data :  
    (1) main lists 내용 : a_001_1_4 0
                        camDir_id_task_score ID

In [1]:
import numpy as np
import os
import sys
import glob
from utils import *

folder_main = '/home/hyoyeonlee/bbs_project/data/main_list/'
fnames_main = glob.glob(folder_main+"*.txt")
fnames_main.sort()
fnames_main


#folder_data = '/home/hyoyeonlee/bbs_project/data/20230201_npy_color2D_common_joints/'
folder_data = '/home/hyoyeonlee/bbs_project/data/20230508_BT_color2D_13joints/'
fnames_data = glob.glob(folder_data+"*/*/*/*/*")
fnames_data.sort()
fnames_data[0]


folder_data_processed1 = '/home/hyoyeonlee/bbs_project/data_processed/_1_coor_transformed/'

/usr/local/lib/python3.8/dist-packages/scipy/__init__.py:143: UserWarning: A NumPy version >=1.19.5 and <1.27.0 is required for this version of SciPy (detected version 1.17.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# main list에서 bad, -1로 표기된 데이터는 제외

- 전체에 비해 bad+neg1 갯수가 1.6%정도임.


- fnames_data : folder_data/camDir/task/score/id/camDir_id_task_score_ID.npy
- main list :   camDir_id_task_score id

In [2]:
countbad=0
countneg1=0
tot=0
info_main = {}

for i in range(len(fnames_main)):
    f = open(fnames_main[i],'r')
    lines = f.readlines()
    tot+=len(lines)
    for j in range(len(lines)):
        ID = lines[j].split(' ')[1]
        if ID=='bad':countbad+=1
        elif ID=='-1':countneg1+=1
        else: info_main[lines[j].split(' ')[0]]=ID
print(tot,countbad,countneg1)

12020 131 62


# main list에서 main의 ID에 해당하는 데이터만 추출

In [3]:
fnames_data_mainOnly = []
count_notFound = 0
for i in range(len(fnames_data)):
    file_info = fnames_data[i].split('/')[-1].split('.')[0][:-2]
    file_info_ID  = fnames_data[i].split('/')[-1].split('.')[0][-1]
    try:
        mainID = info_main[file_info]
        if mainID==file_info_ID:
            fnames_data_mainOnly.append(fnames_data[i])
    except:
        count_notFound+=1
    
print(count_notFound)

102


# main ID해당하는 npy파일 로딩, 좌표변환 (zero-filled data removed)


## $(x_{new},y_{new}) = (\frac{x-xc}{L},\frac{y-yc}{L})$


- 원점 : (xc,yc) = ($\overline{x_{hip}}$,$\overline{y_{hip}}$)


- 단위길이 : L=$\sqrt{(\overline{x_a}-\overline{x_b})^2+(\overline{y_a}-\overline{y_b})^2}$


- where $\overline{x_{joint}} = \dfrac{1}{2}(joint\_L\_x+joint\_R\_x)$


* task 01~10 : a,b = knee, ankle
* task 11~14 : a,b = shoulder, hip

In [4]:
dtype=get_dtypes("COMMON",False)


=============== [dtype for npy] ===============
('frame', <class 'int'>)
('wrist_L_x', <class 'float'>)
('wrist_L_y', <class 'float'>)
('elbow_L_x', <class 'float'>)
('elbow_L_y', <class 'float'>)
('shoulder_L_x', <class 'float'>)
('shoulder_L_y', <class 'float'>)
('shoulder_R_x', <class 'float'>)
('shoulder_R_y', <class 'float'>)
('elbow_R_x', <class 'float'>)
('elbow_R_y', <class 'float'>)
('wrist_R_x', <class 'float'>)
('wrist_R_y', <class 'float'>)
('nose_x', <class 'float'>)
('nose_y', <class 'float'>)
('ankle_L_x', <class 'float'>)
('ankle_L_y', <class 'float'>)
('knee_L_x', <class 'float'>)
('knee_L_y', <class 'float'>)
('hip_L_x', <class 'float'>)
('hip_L_y', <class 'float'>)
('hip_R_x', <class 'float'>)
('hip_R_y', <class 'float'>)
('knee_R_x', <class 'float'>)
('knee_R_y', <class 'float'>)
('ankle_R_x', <class 'float'>)
('ankle_R_y', <class 'float'>)



In [5]:
for i in range(len(fnames_data_mainOnly)):
    fname = fnames_data_mainOnly[i]
    f = np.load(fname)    
    fname_info = fname.split('/')[-1].split('_')
    camDir   = fname_info[0]
    id_scene = fname_info[1]
    task     = fname_info[2]
    score    = fname_info[3]
    ID       = fname_info[-1][0]
    nframes = len(f)
    output = np.zeros(nframes,dtype=dtype)
    for j in range(nframes):
        xc = np.mean([f[j]['hip_L_x'],f[j]['hip_R_x']])
        yc = np.mean([f[j]['hip_L_y'],f[j]['hip_R_y']])
        if int(task)<=10:
            xa = np.mean([f[j]['knee_L_x'],f[j]['knee_R_x']])
            ya = np.mean([f[j]['knee_L_y'],f[j]['knee_R_y']])
            xb = np.mean([f[j]['ankle_L_x'],f[j]['ankle_R_x']])
            yb = np.mean([f[j]['ankle_L_y'],f[j]['ankle_R_y']])
        else:
            xa = np.mean([f[j]['shoulder_L_x'],f[j]['shoulder_R_x']])
            ya = np.mean([f[j]['shoulder_L_y'],f[j]['shoulder_R_y']])
            xb = np.mean([f[j]['hip_L_x'],f[j]['hip_R_x']])
            yb = np.mean([f[j]['hip_L_y'],f[j]['hip_R_y']])
        L = np.sqrt((xa-xb)**2+(ya-yb)**2)
        
        # 스켈레톤 안잡힌 경우 모든 좌표가 0이어서 L=0나오는 경우 -> 저장 안하고 스킵.
        if L==0:
            continue
        
        output[j]['frame'] = f[j]['frame']
        for item in dtype.names:
            if 'x' in item :
                output[j][item] = (f[j][item]-xc)/L 
            elif 'y' in item:
                output[j][item] = (f[j][item]-yc)/L
    out_subfolder = folder_data_processed1+camDir+'/'+task+'/'+score+'/'+id_scene+'/'
    if (os.path.isdir(out_subfolder) is False):
        os.makedirs(out_subfolder)
    
    out_subfolder = out_subfolder+ID+'/'
    if (os.path.isdir(out_subfolder) is False):
        os.makedirs(out_subfolder)
    np.save(out_subfolder+fname.split('/')[-1],output)     